In [1]:
import pandas as pd
import numpy as np
import optuna
import dacon_law_class as dlc
from dacon_law_class import SimpleOps as so

import lightgbm as lgb
import re
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from xgboost.sklearn import XGBClassifier as xgb
# import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')


from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

 ___________________________
|                           |
|======== YearDream ========|
|===========================|
|==== DLC Well Imported ====|
|===========================|
|========= BYJASON =========|
|___________________________|



In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

FileNotFoundError: [Errno 2] No such file or directory: './train.csv'

In [3]:
# train
# test
sample_submission

,ID,first_party_winner
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
1235,TEST_1235,0
1236,TEST_1236,0
1237,TEST_1237,0
1238,TEST_1238,0


In [4]:
train.info()
print('\n\n\n')
# test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  2478 non-null   object
 1   first_party         2478 non-null   object
 2   second_party        2478 non-null   object
 3   facts               2478 non-null   object
 4   first_party_winner  2478 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 96.9+ KB






In [5]:
df = pd.DataFrame(train['facts'])
df = dlc.law_preprocessor(df, 'facts')
train['facts'] = df['facts']
df = pd.DataFrame(train['first_party'])
df = dlc.law_preprocessor(df, 'first_party')
train['first_party'] = df['first_party']
df = pd.DataFrame(train['second_party'])
df = dlc.law_preprocessor(df, 'second_party')
train['second_party'] = df['second_party']
train_cleansed = train.drop(columns='ID')
train_cleansed


,first_party,second_party,facts,first_party_winner
0,phil amant,herman ompson,phil amant caidate for public office made tele...,1
1,ephen duncan,lawrence owens,ramon nelson was ridin his bike when he suffer...,0
2,billy joe mawood,tony patterson waen et al,an alabama ate court convicted billy joe mawoo...,1
3,linkletter,walker,victor linkletter was convicted in ate court o...,0
4,william earl fikes,alabama,in selma alabama an intruder broke into apartm...,1
...,...,...,...,...
2473,hollyfrontier cheyenne refinin llc et al,renewable fuels association et al,conress ameed clean air act rouh enery policy ...,1
2474,rupo mexicano de desarrollo,alliance bo fu inc,alliance bo fu inc an invement fu purchased ap...,1
2475,peuero,united ates,in dirict court sentenced manuel peuero to mon...,0
2476,iiration naturalization service,cyr,enrico cyr lawful permanent resident pled uilt...,0


In [6]:
df = pd.DataFrame(test['facts'])
df = dlc.law_preprocessor(df, 'facts')
test['facts'] = df['facts']
df = pd.DataFrame(test['first_party'])
df = dlc.law_preprocessor(df, 'first_party')
test['first_party'] = df['first_party']
df = pd.DataFrame(test['second_party'])
df = dlc.law_preprocessor(df, 'second_party')
test['second_party'] = df['second_party']
test_cleansed = test.drop(columns='ID')
test_cleansed

,first_party,second_party,facts
0,salerno,united ates,bail reform act allowed federal courts to deta...
1,milber weiss bershad hynes lerach,lexecon inc,lexecon inc was defeant in class action lawsui...
2,no title federal counications coission et al,fox television ations inc et al,in fox television ations broadca billboa music...
3,harold kaufman,united ates,durin his trial for armed robbery of federally...
4,berer,hanlon,in mairate jude issued warrant auorizin search...
...,...,...,...
1235,haitian centers council inc et al,chris sale actin coissioner iiration naturaliz...,accoin to executive oer no sined by president ...
1236,whitman,american truckin associations inc,section of clean air act requires environmenta...
1237,lia matteo john madian,william barr,lia matteo john madian created plan for utiliz...
1238,washinton ate apple advertisin coission,hunt,in nor carolina boa of ariculture adopted reul...


## BERT

@article{turc2019,
  title={Well-Read Students Learn Better: On the Importance of Pre-training Compact Models},
  author={Turc, Iulia and Chang, Ming-Wei and Lee, Kenton and Toutanova, Kristina},
  journal={arXiv preprint arXiv:1908.08962v2 },
  year={2019}
}


In [ ]:
train_facts = pd.DataFrame(test_cleansed['facts'])
train_facts

In [ ]:
test_fact = pd.DataFrame(test_cleansed['facts'])
test_fact

model_input_df = dlc.bert_tokenizer(train_facts, 'facts')

model_input_df.info()

df_part_1, df_part_2, df_part_3, df_part_4, df_part_5, df_part_6, df_part_7, df_part_8, df_part_9, df_part_10, df_part_11, df_part_12, df_part_13, df_part_14, df_part_15, df_part_16, df_part_17, df_part_18, df_part_19, df_part_20, df_part_21, df_part_22, df_part_23, df_part_24, df_part_25, df_part_26 = so.df_divider(train_cleansed, 'facts')
df_part_1 = pd.DataFrame(df_part_1)
df_part_2 = pd.DataFrame(df_part_2)
df_part_3 = pd.DataFrame(df_part_3)
df_part_4 = pd.DataFrame(df_part_4)
df_part_5 = pd.DataFrame(df_part_5)
df_part_6 = pd.DataFrame(df_part_6)
df_part_7 = pd.DataFrame(df_part_7)
df_part_8 = pd.DataFrame(df_part_8)
df_part_9 = pd.DataFrame(df_part_9)
df_part_10 = pd.DataFrame(df_part_10)
df_part_11 = pd.DataFrame(df_part_11)
df_part_12 = pd.DataFrame(df_part_12)
df_part_13 = pd.DataFrame(df_part_13)
df_part_14 = pd.DataFrame(df_part_14)
df_part_15 = pd.DataFrame(df_part_15)
df_part_16 = pd.DataFrame(df_part_16)
df_part_17 = pd.DataFrame(df_part_17)
df_part_18 = pd.DataFrame(df_part_18)
df_part_19 = pd.DataFrame(df_part_19)
df_part_20 = pd.DataFrame(df_part_20)
df_part_21 = pd.DataFrame(df_part_21)
df_part_22 = pd.DataFrame(df_part_22)
df_part_23 = pd.DataFrame(df_part_23)
df_part_24 = pd.DataFrame(df_part_24)
df_part_25 = pd.DataFrame(df_part_25)
df_part_26 = pd.DataFrame(df_part_26)
print(df_part_1)

embedded_df_1 = dlc.auto_tokenizer(train_cleansed, 'facts')
embedded_df_1

embedded_df_1 = embedded_df_1.rename(columns={0:'facts_berted'})

embedded_df_1.to_csv('./embeddings/facts_embedded.csv', index=False)

In [ ]:
first_party_berted = dlc.auto_tokenizer(train_cleansed, 'first_party')
first_party_berted = first_party_berted.rename(columns={0:'first_party_berted'})
first_party_berted.to_csv('./embeddings/first_party_berted.csv', index=False)

In [ ]:
second_party_berted = dlc.auto_tokenizer(train_cleansed, 'second_party')
second_party_berted = second_party_berted.rename(columns={0:'second_party_berted'})
second_party_berted.to_csv('./embeddings/second_party_berted.csv', index=False)

In [ ]:
facts_berted = dlc.auto_tokenizer(train_cleansed, 'facts')

In [ ]:
facts_berted = facts_berted.rename(columns={0:'facts_berted'})
facts_berted.to_csv('./embeddings/facts_berted.csv', index=False)

In [ ]:
first_party_berted = pd.read_csv('./embeddings/first_party_berted.csv')
second_party_berted = pd.read_csv('./embeddings/second_party_berted.csv')
facts_berted = pd.read_csv('./embeddings/facts_berted.csv')

In [ ]:
all_ready_to_ml = pd.concat([first_party_berted['first_party_berted'], second_party_berted['second_party_berted'], facts_berted['facts_berted'], train_cleansed['first_party_winner']], axis=1)
all_ready_to_ml.to_csv('./embeddings/1_train_ready_to_ml.csv', index=False)


In [ ]:
test_first_party_berted = dlc.auto_tokenizer(test_cleansed, 'first_party')
test_first_party_berted = test_first_party_berted.rename(columns={0:'first_party_berted'})
test_first_party_berted.to_csv('./embeddings/test_first_party_berted.csv', index=False)

In [ ]:
test_second_party_berted = dlc.auto_tokenizer(test_cleansed, 'second_party')
test_second_party_berted = test_second_party_berted.rename(columns={0:'second_party_berted'})
test_second_party_berted.to_csv('./embeddings/test_second_party_berted.csv', index=False)

In [ ]:
test_facts_berted = dlc.auto_tokenizer(test_cleansed, 'facts')
test_facts_berted = test_facts_berted.rename(columns={0:'facts_berted'})
test_facts_berted.to_csv('./embeddings/test_facts_berted.csv', index=False)

In [ ]:
test_first_party_berted = pd.read_csv('./embeddings/test_first_party_berted.csv')
test_second_party_berted = pd.read_csv('./embeddings/test_second_party_berted.csv')
test_facts_berted = pd.read_csv('./embeddings/test_facts_berted.csv')

In [ ]:
test_ready_to_ml = pd.concat([test_first_party_berted['first_party_berted'], test_second_party_berted['second_party_berted'], test_facts_berted['facts_berted']], axis=1)
test_ready_to_ml.to_csv('./embeddings/2_test_ready_to_ml.csv', index=False)

# 여기

In [7]:
train_to_ml = pd.read_csv('./embeddings/1_train_ready_to_ml.csv')
test_ready_to_ml = pd.read_csv('./embeddings/2_test_ready_to_ml.csv')

In [8]:
train_to_ml_fp_pr = dlc.tensor_separator(train_to_ml, 'first_party_berted')
train_to_ml_sp_pr = dlc.tensor_separator(train_to_ml, 'second_party_berted')
train_to_ml_facts_pr = dlc.tensor_separator(train_to_ml, 'facts_berted')

100%|██████████████████████| 2478/2478 [00:01<00:00, 2242.07it/s]


In [9]:
train_to_ml_fp_pr = train_to_ml_fp_pr.astype('float64')
train_to_ml_sp_pr = train_to_ml_sp_pr.astype('float64') 
train_to_ml_facts_pr = train_to_ml_facts_pr.astype('float64')


In [10]:
train_to_ml_fp_pr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Columns: 768 entries, 0 to 767
dtypes: float64(768)
memory usage: 14.5 MB


In [11]:
test_to_ml_fp_pr = dlc.tensor_separator(test_ready_to_ml, 'first_party_berted')
test_to_ml_sp_pr = dlc.tensor_separator(test_ready_to_ml, 'second_party_berted')
test_to_ml_facts_pr = dlc.tensor_separator(test_ready_to_ml, 'facts_berted')

100%|██████████████████████| 1240/1240 [00:00<00:00, 2250.22it/s]


In [12]:
test_to_ml_fp_pr = test_to_ml_fp_pr.astype('float64')
test_to_ml_sp_pr = test_to_ml_sp_pr.astype('float64')
test_to_ml_facts_pr = test_to_ml_facts_pr.astype('float64')

In [13]:
to_be_X = pd.concat([train_to_ml_fp_pr, train_to_ml_facts_pr], axis=1)
to_be_test_x = pd.concat([test_to_ml_fp_pr, test_to_ml_facts_pr], axis=1)


In [14]:
to_be_X.columns = np.arange(len(to_be_X.columns))
to_be_X = pd.concat([to_be_X, train_to_ml['first_party_winner']], axis =1)
to_be_X

,0,1,2,3,4,5,6,7,8,9,...,1527,1528,1529,1530,1531,1532,1533,1534,1535,first_party_winner
0,0.005036,0.006439,-0.012888,-0.007303,-0.017571,-0.029552,-0.006318,0.025047,-0.009394,-0.015124,...,-0.018174,0.001737,0.003518,-0.010968,-0.030724,-0.035971,-0.038689,-0.006275,-0.010507,1
1,-0.003902,0.026781,-0.044628,-0.032344,-0.001729,-0.018735,0.002111,-0.018207,-0.009919,-0.075807,...,-0.010969,0.006232,-0.012417,-0.020754,-0.016356,-0.039208,-0.031188,-0.014293,-0.026212,0
2,0.003050,0.009259,-0.017064,-0.019458,-0.006200,-0.008553,0.050010,0.035903,-0.030393,-0.020106,...,-0.011412,-0.006155,-0.010546,-0.024606,-0.025278,-0.054787,-0.016296,-0.009530,-0.012053,1
3,-0.015234,-0.000875,0.015897,-0.018534,0.054608,-0.030493,0.041842,0.057857,-0.013273,-0.019490,...,-0.019637,-0.005936,-0.003335,-0.026111,-0.015403,0.000960,-0.027738,0.005133,-0.012935,0
4,-0.011956,-0.009987,0.020198,0.009508,0.024592,-0.037784,0.020635,-0.023835,-0.029138,-0.031670,...,-0.033741,0.007560,-0.023639,-0.028438,-0.010833,-0.030627,-0.035033,-0.000732,-0.024343,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,-0.020591,-0.056972,-0.033879,0.031382,0.079985,0.025763,-0.006569,0.044251,-0.008797,-0.060979,...,-0.027422,0.000112,0.015122,0.009355,-0.046905,-0.002683,-0.008214,-0.000282,-0.009470,1
2474,-0.023678,-0.022937,-0.022214,0.013166,0.054840,0.011287,0.016339,0.090016,0.000314,0.001474,...,-0.017667,0.015822,-0.002527,0.004892,-0.039568,-0.052614,-0.034712,-0.004466,0.008349,1
2475,-0.004295,-0.029256,-0.015316,-0.010641,0.007141,0.021234,0.005180,0.064935,-0.022453,-0.034121,...,-0.011819,0.019265,-0.019696,-0.024496,-0.042203,-0.044682,-0.023756,-0.004682,0.003538,0
2476,0.021019,-0.003120,-0.007031,0.007962,0.045658,0.014659,0.024269,-0.021894,-0.032928,-0.029230,...,-0.023799,0.002132,-0.011824,-0.020932,-0.033988,-0.028924,-0.045331,-0.013245,-0.016124,0


In [15]:
to_be_test_x.columns = np.arange(len(to_be_test_x.columns))
to_be_test_x

,0,1,2,3,4,5,6,7,8,9,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,-0.006672,-0.080011,-0.014580,-0.038840,0.048455,-0.040639,0.039663,0.068547,0.002239,-0.029425,...,-0.012600,-0.023597,-0.003138,-0.008353,-0.034796,-0.020804,-0.006913,-0.038063,0.008657,-0.021999
1,-0.003053,-0.011769,-0.034803,-0.041324,-0.026498,-0.014561,0.012917,-0.009658,-0.000563,-0.018914,...,-0.018383,-0.027521,0.004810,-0.006279,-0.019910,-0.044318,-0.049594,-0.022413,0.012250,0.003738
2,-0.025638,-0.016005,-0.021478,0.014413,0.048054,-0.036683,-0.034054,0.043429,-0.017806,-0.045973,...,-0.023804,-0.017397,0.008190,-0.010989,-0.014011,-0.041264,-0.009615,-0.012293,-0.000732,0.010219
3,0.028991,0.036599,-0.059128,-0.030722,-0.017587,-0.028222,0.052064,-0.009327,0.006705,-0.008745,...,-0.021711,-0.015069,-0.001111,-0.014473,-0.025876,-0.006428,-0.018803,-0.028281,-0.004001,-0.012030
4,0.036941,-0.039247,-0.026675,-0.006587,-0.022472,-0.016113,0.046348,-0.012971,0.029547,-0.051456,...,-0.016862,-0.034918,0.005033,0.000040,-0.037341,-0.040531,-0.014745,-0.042890,-0.000467,-0.010147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,-0.017606,-0.026507,-0.038503,-0.013017,0.036128,-0.006750,-0.023582,0.004234,-0.010011,-0.024067,...,-0.010572,-0.034447,-0.002972,-0.012302,-0.004949,-0.046846,-0.039191,-0.034175,-0.013973,0.006327
1236,-0.001220,-0.002364,-0.025906,-0.023937,0.002847,-0.021732,0.076602,-0.014772,-0.012813,-0.014318,...,-0.000953,-0.013990,0.002872,-0.030309,-0.013520,-0.053392,-0.012986,-0.014506,-0.011636,0.024630
1237,0.001026,0.023440,0.013377,0.016586,0.024960,-0.001395,0.063505,-0.017101,-0.032935,-0.016052,...,-0.018592,-0.016234,0.018842,-0.002355,-0.015415,-0.033917,-0.057270,-0.028927,-0.005387,0.000249
1238,0.001634,-0.000814,-0.014695,0.003300,0.001420,0.028769,-0.025222,0.065449,-0.011581,-0.044858,...,0.004752,0.003640,0.013376,-0.006307,-0.005089,-0.044533,-0.016824,-0.018459,-0.004400,0.022938


In [16]:
X = to_be_X.drop(columns='first_party_winner')
y = pd.DataFrame(to_be_X['first_party_winner'])

In [17]:
test_x = to_be_test_x

In [18]:
X.info()

print('\n --------------------------------------- \n')

y.info()

print('\n --------------------------------------- \n')

test_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Columns: 1536 entries, 0 to 1535
dtypes: float64(1536)
memory usage: 29.0 MB

 --------------------------------------- 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 1 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   first_party_winner  2478 non-null   int64
dtypes: int64(1)
memory usage: 19.5 KB

 --------------------------------------- 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1240 entries, 0 to 1239
Columns: 1536 entries, 0 to 1535
dtypes: float64(1536)
memory usage: 14.5 MB


In [19]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(1734, 1536) (744, 1536) (1734, 1) (744, 1)


In [ ]:
def xgb_objective(trial):
    xgb_params = {
        'objective': 'binary:logistic',
        'eval_metric': 'error',
        'booster': 'gbtree',
        'nthread': trial.suggest_int('nthread', 1, 15),
        'n_estimators' : trial.suggest_int('n_estimators', 25, 1000),
        'max_depth': trial.suggest_int('max_depth', 4, 15),
        'subsample': trial.suggest_uniform('subsample', 0.1, 0.3),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 0.7),
        'lambda': trial.suggest_loguniform('lambda', 0.2, 200),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 3)
    }
    


    xgb_model = xgb(**xgb_params)
    xgb_model.fit(X_train, y_train)
    xgb_preds = xgb_model.predict(X_val)
    
    return accuracy_score(y_val, xgb_preds)


In [ ]:
def lgb_objective(trial):
    lgb_params = {
        'application': 'binary',
        'max_depth': -1,
        'metric': 'binary_logloss',
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt',  'dart']),
        'num_leaves': trial.suggest_int('num_leaves', 10, 2000),
        'lambda' : trial.suggest_float('lambda', 0.01, 0.5),
        'num_iteration': 500,
        'n_jobs': -1,
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.1),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.7, 0.9),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 0.8),
        'random_state': 42
    }
    
    lgb_model = lgb.LGBMClassifier(**lgb_params)
    lgb_model.fit(X_train, y_train)
    lgb_preds = lgb_model.predict(X_val)
    
    return accuracy_score(y_val, lgb_preds)


In [ ]:
def cat_objective(trial):
    params = {
            'loss_function': 'Logloss',
            'learning_rate': learning_rate,
            'depth': trial.suggest_int('depth', 3, 10),
            'random_state': 42
        }


    model = cat.CatBoostClassifier(**params)
    model.fit(X_train, y_train)
    pred = model.predict(X_val)
    
    return accuracy_score(y_val, pred)

In [91]:
def model_selector(trial):
    model_type = trial.suggest_categorical('model_type', ['xgb', 'lgbm'])
    random_state = 42


    if model_type == 'xgb':
        eval_metric = 'error'
        objective = 'binary:logistic'
        tree_method = trial.suggest_categorical('tree_method', ['exact', 'approx', 'hist'])

        if tree_method == 'exact':
            sampling_method = 'uniform'
            subsample = 0.5
            booster = trial.suggest_categorical('booster', ['dart', 'gbtree'])
            if booster == 'gbtree':
                max_depth = trial.suggest_int('max_depth', 1, 300)
                n_estimators = trial.suggest_int('n_estimators', 1, 1000)
                if n_estimators < 500:
                    colsample_bytree = trial.suggest_loguniform('colsample_bytree', 0.3, 0.9)
                    learning_rate = trial.suggest_categorical('learning_rate', [1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2,
                                                                                1e-1])  # aka eta, from 0 to 1, loguniform
                    if learning_rate in [1e-4, 5e-4]:
                        min_split_loss = trial.suggest_loguniform('min_split_loss', 0.1,
                                                                  500)  # aka gamma, from 0 to 1, loguniform
                        min_child_weight = trial.suggest_int('min_child_weight', 1, 20)  # from 0 to infinite, int
                        alpha = trial.suggest_loguniform('alpha', 0.1, 20)  # from 0 to infinite, loguniform
                        reg_lambda = trial.suggest_loguniform('reg_lambda', 0.01, 20)  # from 0 to infinite, loguniform
                        model = xgb(
                            eval_metric=eval_metric,
                            objective=objective,
                            learning_rate=learning_rate,
                            sampling_method=sampling_method,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree,
                            max_depth=max_depth,
                            tree_method=tree_method,
                            booster=booster,
                            min_split_loss=min_split_loss,
                            min_child_weight=min_child_weight,
                            alpha=alpha,
                            reg_lambda=reg_lambda,
                            random_state=random_state
                        )

                        model.fit(X_train, y_train)
                        preds = model.predict(X_val)

                        return accuracy_score(y_val, preds)

                    elif learning_rate in [1e-3, 5e-3, 1e-2]:
                        min_split_loss = trial.suggest_loguniform('min_split_loss', 200,
                                                                  700)  # aka gamma, from 0 to 1, loguniform
                        min_child_weight = trial.suggest_int('min_child_weight', 10, 30)  # from 0 to infinite, int
                        alpha = trial.suggest_loguniform('alpha', 10, 30)  # from 0 to infinite, loguniform
                        reg_lambda = trial.suggest_loguniform('reg_lambda', 10, 30)  # from 0 to infinite, loguniform
                        model = xgb(
                            eval_metric=eval_metric,
                            objective=objective,
                            learning_rate=learning_rate,
                            sampling_method=sampling_method,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree,
                            max_depth=max_depth,
                            tree_method=tree_method,
                            booster=booster,
                            min_split_loss=min_split_loss,
                            min_child_weight=min_child_weight,
                            alpha=alpha,
                            reg_lambda=reg_lambda,
                            random_state=random_state
                        )

                        model.fit(X_train, y_train)
                        preds = model.predict(X_val)

                        return accuracy_score(y_val, preds)

                    elif learning_rate in [5e-2, 1e-1]:
                        min_split_loss = trial.suggest_loguniform('min_split_loss', 500,
                                                                  1000)  # aka gamma, from 0 to 1, loguniform
                        min_child_weight = trial.suggest_int('min_child_weight', 20, 40)  # from 0 to infinite, int
                        alpha = trial.suggest_loguniform('alpha', 20, 40)  # from 0 to infinite, loguniform
                        reg_lambda = trial.suggest_loguniform('reg_lambda', 20, 40)  # from 0 to infinite, loguniform
                        model = xgb(
                            eval_metric=eval_metric,
                            objective=objective,
                            learning_rate=learning_rate,
                            sampling_method=sampling_method,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree,
                            max_depth=max_depth,
                            tree_method=tree_method,
                            booster=booster,
                            min_split_loss=min_split_loss,
                            min_child_weight=min_child_weight,
                            alpha=alpha,
                            reg_lambda=reg_lambda,
                            random_state=random_state
                        )
                        model.fit(X_train, y_train)
                        preds = model.predict(X_val)

                        return accuracy_score(y_val, preds)

                elif n_estimators > 500:
                    colsample_bytree = trial.suggest_loguniform('colsample_bytree', 0.1, 0.7)
                    learning_rate = trial.suggest_categorical('learning_rate', [1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2,
                                                                            1e-1])  # aka eta, from 0 to 1, loguniform
                    if learning_rate in [1e-4, 5e-4]:
                        min_split_loss = trial.suggest_loguniform('min_split_loss', 0.1,
                                                                  500)  # aka gamma, from 0 to 1, loguniform
                        min_child_weight = trial.suggest_int('min_child_weight', 1, 20)  # from 0 to infinite, int
                        alpha = trial.suggest_loguniform('alpha', 0.1, 20)  # from 0 to infinite, loguniform
                        reg_lambda = trial.suggest_loguniform('reg_lambda', 0.01, 20)  # from 0 to infinite, loguniform
                        model = xgb(
                            eval_metric=eval_metric,
                            objective=objective,
                            learning_rate=learning_rate,
                            sampling_method=sampling_method,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree,
                            max_depth=max_depth,
                            tree_method=tree_method,
                            booster=booster,
                            min_split_loss=min_split_loss,
                            min_child_weight=min_child_weight,
                            alpha=alpha,
                            reg_lambda=reg_lambda,
                            random_state=random_state
                        )

                        model.fit(X_train, y_train)
                        preds = model.predict(X_val)

                        return accuracy_score(y_val, preds)

                    elif learning_rate in [1e-3, 5e-3, 1e-2]:
                        min_split_loss = trial.suggest_loguniform('min_split_loss', 200,
                                                                  700)  # aka gamma, from 0 to 1, loguniform
                        min_child_weight = trial.suggest_int('min_child_weight', 10, 30)  # from 0 to infinite, int
                        alpha = trial.suggest_loguniform('alpha', 10, 30)  # from 0 to infinite, loguniform
                        reg_lambda = trial.suggest_loguniform('reg_lambda', 10, 30)  # from 0 to infinite, loguniform
                        model = xgb(
                            eval_metric=eval_metric,
                            objective=objective,
                            learning_rate=learning_rate,
                            sampling_method=sampling_method,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree,
                            max_depth=max_depth,
                            tree_method=tree_method,
                            booster=booster,
                            min_split_loss=min_split_loss,
                            min_child_weight=min_child_weight,
                            alpha=alpha,
                            reg_lambda=reg_lambda,
                            random_state=random_state
                        )

                        model.fit(X_train, y_train)
                        preds = model.predict(X_val)

                        return accuracy_score(y_val, preds)

                    elif learning_rate in [5e-2, 1e-1]:
                        min_split_loss = trial.suggest_loguniform('min_split_loss', 500,
                                                                  1000)  # aka gamma, from 0 to 1, loguniform
                        min_child_weight = trial.suggest_int('min_child_weight', 20, 40)  # from 0 to infinite, int
                        alpha = trial.suggest_loguniform('alpha', 20, 40)  # from 0 to infinite, loguniform
                        reg_lambda = trial.suggest_loguniform('reg_lambda', 20, 40)  # from 0 to infinite, loguniform
                        model = xgb(
                            eval_metric=eval_metric,
                            objective=objective,
                            learning_rate=learning_rate,
                            sampling_method=sampling_method,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree,
                            max_depth=max_depth,
                            tree_method=tree_method,
                            booster=booster,
                            min_split_loss=min_split_loss,
                            min_child_weight=min_child_weight,
                            alpha=alpha,
                            reg_lambda=reg_lambda,
                            random_state=random_state
                        )

                        model.fit(X_train, y_train)
                        preds = model.predict(X_val)

                        return accuracy_score(y_val, preds)

            elif booster == 'dart':
                max_depth = trial.suggest_int('max_depth', 1, 300)
                colsample_bytree = trial.suggest_loguniform('colsample_bytree', 0.5, 0.9)
                learning_rate = trial.suggest_categorical('learning_rate', [1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2,
                                                                            1e-1])  # aka eta, from 0 to 1, loguniform
                if learning_rate in [1e-4, 5e-4]:
                    rate_drop = trial.suggest_loguniform('rate_drop', 0.01, 0.5)
                    min_child_weight = trial.suggest_int('min_child_weight', 1, 20)  # from 0 to infinite, int
                    model = xgb(
                        eval_metric=eval_metric,
                        objective=objective,
                        learning_rate=learning_rate,
                        sampling_method=sampling_method,
                        subsample=subsample,
                        colsample_bytree=colsample_bytree,
                        rate_drop=rate_drop,
                        max_depth=max_depth,
                        tree_method=tree_method,
                        booster=booster,
                        min_child_weight=min_child_weight,
                        random_state=random_state
                    )

                    model.fit(X_train, y_train)
                    preds = model.predict(X_val)

                    return accuracy_score(y_val, preds)

                elif learning_rate in [1e-3, 5e-3, 1e-2]:
                    rate_drop = trial.suggest_loguniform('rate_drop', 0.3, 0.7)
                    min_child_weight = trial.suggest_int('min_child_weight', 10, 30)  # from 0 to infinite, int
                    model = xgb(
                        eval_metric=eval_metric,
                        objective=objective,
                        learning_rate=learning_rate,
                        sampling_method=sampling_method,
                        subsample=subsample,
                        colsample_bytree=colsample_bytree,
                        rate_drop=rate_drop,
                        max_depth=max_depth,
                        tree_method=tree_method,
                        booster=booster,
                        min_child_weight=min_child_weight,
                        random_state=random_state
                    )

                    model.fit(X_train, y_train)
                    preds = model.predict(X_val)

                    return accuracy_score(y_val, preds)

                elif learning_rate in [5e-2, 1e-1]:
                    rate_drop = trial.suggest_loguniform('rate_drop', 0.5, 1.0)
                    min_child_weight = trial.suggest_int('min_child_weight', 20, 40)  # from 0 to infinite, int
                    model = xgb(
                        eval_metric=eval_metric,
                        objective=objective,
                        learning_rate=learning_rate,
                        sampling_method=sampling_method,
                        subsample=subsample,
                        colsample_bytree=colsample_bytree,
                        rate_drop=rate_drop,
                        max_depth=max_depth,
                        tree_method=tree_method,
                        booster=booster,
                        min_child_weight=min_child_weight,
                        random_state=random_state
                    )

                    model.fit(X_train, y_train)
                    preds = model.predict(X_val)

                    return accuracy_score(y_val, preds)

            # elif booster == 'gblinear':
            #     learning_rate = trial.suggest_categorical('learning_rate', [1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2,
            #                                                                 1e-1])  # aka eta, from 0 to 1, loguniform
            #     if learning_rate in [1e-4, 5e-4]:
            #         alpha = trial.suggest_loguniform('alpha', 0.1, 20)  # from 0 to infinite, loguniform
            #         reg_lambda = trial.suggest_loguniform('reg_lambda', 0.01, 20)  # from 0 to infinite, loguniform
            #         model = xgb(
            #             eval_metric=eval_metric,
            #             objective=objective,
            #             learning_rate=learning_rate,
            #             sampling_method=sampling_method,
            #             subsample=subsample,
            #             tree_method=tree_method,
            #             booster=booster,
            #             alpha=alpha,
            #             reg_lambda=reg_lambda,
            #             random_state=random_state
            #         )
            #
            #         model.fit(X_train, y_train)
            #         preds = model.predict(X_val)
            #
            #         return accuracy_score(y_val, preds)
            #
            #     elif learning_rate in [1e-3, 5e-3, 1e-2]:
            #         alpha = trial.suggest_loguniform('alpha', 10, 30)  # from 0 to infinite, loguniform
            #         reg_lambda = trial.suggest_loguniform('reg_lambda', 10, 30)  # from 0 to infinite, loguniform
            #         model = xgb(
            #             eval_metric=eval_metric,
            #             objective=objective,
            #             learning_rate=learning_rate,
            #             sampling_method=sampling_method,
            #             subsample=subsample,
            #             tree_method=tree_method,
            #             booster=booster,
            #             alpha=alpha,
            #             reg_lambda=reg_lambda,
            #             random_state=random_state
            #         )
            #
            #         model.fit(X_train, y_train)
            #         preds = model.predict(X_val)
            #
            #         return accuracy_score(y_val, preds)
            #
            #     elif learning_rate in [5e-2, 1e-1]:
            #         alpha = trial.suggest_loguniform('alpha', 20, 40)  # from 0 to infinite, loguniform
            #         reg_lambda = trial.suggest_loguniform('reg_lambda', 20, 40)  # from 0 to infinite, loguniform
            #         model = xgb(
            #             eval_metric=eval_metric,
            #             objective=objective,
            #             learning_rate=learning_rate,
            #             sampling_method=sampling_method,
            #             subsample=subsample,
            #             tree_method=tree_method,
            #             booster=booster,
            #             alpha=alpha,
            #             reg_lambda=reg_lambda,
            #             random_state=random_state
            #         )
            #
            #         model.fit(X_train, y_train)
            #         preds = model.predict(X_val)
            #
            #         return accuracy_score(y_val, preds)

        else:
            sampling_method = 'uniform'
            booster = trial.suggest_categorical('booster', ['dart', 'gbtree'])
            if booster == 'gbtree':
                subsample = trial.suggest_loguniform('subsample', 0.1, 0.5)
                max_depth = 0
                n_estimators = trial.suggest_int('n_estimators', 1, 1000)
                if n_estimators < 500:
                    colsample_bytree = trial.suggest_loguniform('colsample_bytree', 0.3, 0.9)
                    learning_rate = trial.suggest_categorical('learning_rate', [1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2,
                                                                                1e-1])  # aka eta, from 0 to 1, loguniform
                    if learning_rate in [1e-4, 5e-4]:
                        min_split_loss = trial.suggest_loguniform('min_split_loss', 0.1,
                                                                  400)  # aka gamma, from 0 to 1, loguniform
                        min_child_weight = trial.suggest_int('min_child_weight', 1, 20)  # from 0 to infinite, int
                        alpha = trial.suggest_loguniform('alpha', 0.1, 20)  # from 0 to infinite, loguniform
                        reg_lambda = trial.suggest_loguniform('reg_lambda', 0.01, 20)  # from 0 to infinite, loguniform
                        model = xgb(
                            eval_metric=eval_metric,
                            objective=objective,
                            learning_rate=learning_rate,
                            sampling_method=sampling_method,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree,
                            max_depth=max_depth,
                            tree_method=tree_method,
                            booster=booster,
                            min_split_loss=min_split_loss,
                            min_child_weight=min_child_weight,
                            alpha=alpha,
                            reg_lambda=reg_lambda,
                            random_state=random_state
                        )

                        model.fit(X_train, y_train)
                        preds = model.predict(X_val)

                        return accuracy_score(y_val, preds)

                    elif learning_rate in [1e-3, 5e-3, 1e-2]:
                        min_split_loss = trial.suggest_loguniform('min_split_loss', 200,
                                                                  700)  # aka gamma, from 0 to 1, loguniform
                        min_child_weight = trial.suggest_int('min_child_weight', 10, 30)  # from 0 to infinite, int
                        alpha = trial.suggest_loguniform('alpha', 10, 30)  # from 0 to infinite, loguniform
                        reg_lambda = trial.suggest_loguniform('reg_lambda', 10, 30)  # from 0 to infinite, loguniform
                        model = xgb(
                            eval_metric=eval_metric,
                            objective=objective,
                            learning_rate=learning_rate,
                            sampling_method=sampling_method,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree,
                            max_depth=max_depth,
                            tree_method=tree_method,
                            booster=booster,
                            min_split_loss=min_split_loss,
                            min_child_weight=min_child_weight,
                            alpha=alpha,
                            reg_lambda=reg_lambda,
                            random_state=random_state
                        )

                        model.fit(X_train, y_train)
                        preds = model.predict(X_val)

                        return accuracy_score(y_val, preds)

                    elif learning_rate in [5e-2, 1e-1]:
                        min_split_loss = trial.suggest_loguniform('min_split_loss', 400,
                                                                  1000)  # aka gamma, from 0 to 1, loguniform
                        min_child_weight = trial.suggest_int('min_child_weight', 20, 40)  # from 0 to infinite, int
                        alpha = trial.suggest_loguniform('alpha', 20, 40)  # from 0 to infinite, loguniform
                        reg_lambda = trial.suggest_loguniform('reg_lambda', 20, 40)  # from 0 to infinite, loguniform
                        model = xgb(
                            eval_metric=eval_metric,
                            objective=objective,
                            learning_rate=learning_rate,
                            sampling_method=sampling_method,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree,
                            max_depth=max_depth,
                            tree_method=tree_method,
                            booster=booster,
                            min_split_loss=min_split_loss,
                            min_child_weight=min_child_weight,
                            alpha=alpha,
                            reg_lambda=reg_lambda,
                            random_state=random_state
                        )

                        model.fit(X_train, y_train)
                        preds = model.predict(X_val)

                        return accuracy_score(y_val, preds)

                elif n_estimators > 500:
                    colsample_bytree = trial.suggest_loguniform('colsample_bytree', 0.1, 0.7)
                    learning_rate = trial.suggest_categorical('learning_rate', [1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2,
                                                                                1e-1])  # aka eta, from 0 to 1, loguniform
                    if learning_rate in [1e-4, 5e-4]:
                        min_split_loss = trial.suggest_loguniform('min_split_loss', 0.1,
                                                                  400)  # aka gamma, from 0 to 1, loguniform
                        min_child_weight = trial.suggest_int('min_child_weight', 1, 20)  # from 0 to infinite, int
                        alpha = trial.suggest_loguniform('alpha', 0.1, 20)  # from 0 to infinite, loguniform
                        reg_lambda = trial.suggest_loguniform('reg_lambda', 0.01, 20)  # from 0 to infinite, loguniform
                        model = xgb(
                            eval_metric=eval_metric,
                            objective=objective,
                            learning_rate=learning_rate,
                            sampling_method=sampling_method,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree,
                            max_depth=max_depth,
                            tree_method=tree_method,
                            booster=booster,
                            min_split_loss=min_split_loss,
                            min_child_weight=min_child_weight,
                            alpha=alpha,
                            reg_lambda=reg_lambda,
                            random_state=random_state
                        )

                        model.fit(X_train, y_train)
                        preds = model.predict(X_val)

                        return accuracy_score(y_val, preds)

                    elif learning_rate in [1e-3, 5e-3, 1e-2]:
                        min_split_loss = trial.suggest_loguniform('min_split_loss', 200,
                                                                  700)  # aka gamma, from 0 to 1, loguniform
                        min_child_weight = trial.suggest_int('min_child_weight', 10, 30)  # from 0 to infinite, int
                        alpha = trial.suggest_loguniform('alpha', 10, 30)  # from 0 to infinite, loguniform
                        reg_lambda = trial.suggest_loguniform('reg_lambda', 10, 30)  # from 0 to infinite, loguniform
                        model = xgb(
                            eval_metric=eval_metric,
                            objective=objective,
                            learning_rate=learning_rate,
                            sampling_method=sampling_method,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree,
                            max_depth=max_depth,
                            tree_method=tree_method,
                            booster=booster,
                            min_split_loss=min_split_loss,
                            min_child_weight=min_child_weight,
                            alpha=alpha,
                            reg_lambda=reg_lambda,
                            random_state=random_state
                        )

                        model.fit(X_train, y_train)
                        preds = model.predict(X_val)

                        return accuracy_score(y_val, preds)

                    elif learning_rate in [5e-2, 1e-1]:
                        min_split_loss = trial.suggest_loguniform('min_split_loss', 400,
                                                                  1000)  # aka gamma, from 0 to 1, loguniform
                        min_child_weight = trial.suggest_int('min_child_weight', 20, 40)  # from 0 to infinite, int
                        alpha = trial.suggest_loguniform('alpha', 20, 40)  # from 0 to infinite, loguniform
                        reg_lambda = trial.suggest_loguniform('reg_lambda', 20, 40)  # from 0 to infinite, loguniform
                        model = xgb(
                            eval_metric=eval_metric,
                            objective=objective,
                            learning_rate=learning_rate,
                            sampling_method=sampling_method,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree,
                            max_depth=max_depth,
                            tree_method=tree_method,
                            booster=booster,
                            min_split_loss=min_split_loss,
                            min_child_weight=min_child_weight,
                            alpha=alpha,
                            reg_lambda=reg_lambda,
                            random_state=random_state
                        )

                        model.fit(X_train, y_train)
                        preds = model.predict(X_val)

                        return accuracy_score(y_val, preds)

            elif booster == 'dart':
                subsample = 0.5
                max_depth = 0
                colsample_bytree = trial.suggest_loguniform('colsample_bytree', 0.5, 0.9)
                learning_rate = trial.suggest_categorical('learning_rate', [1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2,
                                                                            1e-1])  # aka eta, from 0 to 1, loguniform
                if learning_rate in [1e-4, 5e-4]:
                    rate_drop = trial.suggest_loguniform('rate_drop', 0.01, 0.5)
                    min_child_weight = trial.suggest_int('min_child_weight', 1, 20)  # from 0 to infinite, int
                    model = xgb(
                        eval_metric=eval_metric,
                        objective=objective,
                        learning_rate=learning_rate,
                        sampling_method=sampling_method,
                        subsample=subsample,
                        colsample_bytree=colsample_bytree,
                        max_depth=max_depth,
                        tree_method=tree_method,
                        booster=booster,
                        rate_drop=rate_drop,
                        min_child_weight=min_child_weight,
                        random_state=random_state
                    )

                    model.fit(X_train, y_train)
                    preds = model.predict(X_val)

                    return accuracy_score(y_val, preds)


                elif learning_rate in [1e-3, 5e-3, 1e-2]:
                    rate_drop = trial.suggest_loguniform('rate_drop', 0.3, 0.7)
                    min_child_weight = trial.suggest_int('min_child_weight', 10, 30)  # from 0 to infinite, int
                    model = xgb(
                        eval_metric=eval_metric,
                        objective=objective,
                        learning_rate=learning_rate,
                        sampling_method=sampling_method,
                        subsample=subsample,
                        colsample_bytree=colsample_bytree,
                        max_depth=max_depth,
                        tree_method=tree_method,
                        booster=booster,
                        rate_drop=rate_drop,
                        min_child_weight=min_child_weight,
                        random_state=random_state
                    )

                    model.fit(X_train, y_train)
                    preds = model.predict(X_val)

                    return accuracy_score(y_val, preds)

                elif learning_rate in [5e-2, 1e-1]:
                    rate_drop = trial.suggest_loguniform('rate_drop', 0.5, 1.0)
                    min_child_weight = trial.suggest_int('min_child_weight', 20, 40)  # from 0 to infinite, int
                    model = xgb(
                        eval_metric=eval_metric,
                        objective=objective,
                        learning_rate=learning_rate,
                        sampling_method=sampling_method,
                        subsample=subsample,
                        colsample_bytree=colsample_bytree,
                        max_depth=max_depth,
                        tree_method=tree_method,
                        booster=booster,
                        rate_drop=rate_drop,
                        min_child_weight=min_child_weight,
                        random_state=random_state
                    )

                    model.fit(X_train, y_train)
                    preds = model.predict(X_val)

                    return accuracy_score(y_val, preds)


            # elif booster == 'gblinear':
            #     learning_rate = trial.suggest_categorical('learning_rate', [1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2,
            #                                                                 1e-1])  # aka eta, from 0 to 1, loguniform
            #     if learning_rate in [1e-4, 5e-4]:
            #         alpha = trial.suggest_loguniform('alpha', 0.1, 20)  # from 0 to infinite, loguniform
            #         reg_lambda = trial.suggest_loguniform('reg_lambda', 0.01, 20)  # from 0 to infinite, loguniform
            #         model = xgb(
            #             eval_metric=eval_metric,
            #             objective=objective,
            #             learning_rate=learning_rate,
            #             sampling_method=sampling_method,
            #             tree_method=tree_method,
            #             booster=booster,
            #             alpha=alpha,
            #             reg_lambda=reg_lambda,
            #             random_state=random_state
            #         )
            #         model.fit(X_train, y_train)
            #         preds = model.predict(X_val)
            #
            #         return accuracy_score(y_val, preds)
            #
            #     elif learning_rate in [1e-3, 5e-3, 1e-2]:
            #         alpha = trial.suggest_loguniform('alpha', 10, 30)  # from 0 to infinite, loguniform
            #         reg_lambda = trial.suggest_loguniform('reg_lambda', 10, 30)  # from 0 to infinite, loguniform
            #         model = xgb(
            #             eval_metric=eval_metric,
            #             objective=objective,
            #             learning_rate=learning_rate,
            #             sampling_method=sampling_method,
            #             tree_method=tree_method,
            #             booster=booster,
            #             alpha=alpha,
            #             reg_lambda=reg_lambda,
            #             random_state=random_state
            #         )
            #         model.fit(X_train, y_train)
            #         preds = model.predict(X_val)
            #
            #         return accuracy_score(y_val, preds)
            #
            #     elif learning_rate in [5e-2, 1e-1]:
            #         alpha = trial.suggest_loguniform('alpha', 20, 40)  # from 0 to infinite, loguniform
            #         reg_lambda = trial.suggest_loguniform('reg_lambda', 20, 40)  # from 0 to infinite, loguniform
            #         model = xgb(
            #             eval_metric=eval_metric,
            #             objective=objective,
            #             learning_rate=learning_rate,
            #             sampling_method=sampling_method,
            #             tree_method=tree_method,
            #             booster=booster,
            #             alpha=alpha,
            #             reg_lambda=reg_lambda,
            #             random_state=random_state
            #         )
            #         model.fit(X_train, y_train)
            #         preds = model.predict(X_val)
            #
            #         return accuracy_score(y_val, preds)

    elif model_type == 'lgbm':
        objective = 'binary'
        metric = 'accuracy'
        num_threads = 0
        max_depth = -1

        num_leaves = trial.suggest_int('num_leaves', 1, 1000)
        boosting_type = trial.suggest_categorical('boosting_type', ['gbdt', 'dart', 'rf', 'goss'])
        if boosting_type == 'gbdt':
            n_estimators = trial.suggest_int('n_estimators', 1, 500)
            learning_rate = trial.suggest_categorical('learning_rate', [1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2,
                                                                        1e-1])  # aka eta, from 0 to 1, loguniform
            if learning_rate in [1e-4, 5e-4]:
                min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 20)
                feature_fraction = trial.suggest_uniform('feature_fraction', 0.5, 0.999)
                bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.5, 0.999)
                bagging_freq = trial.suggest_int('bagging_freq', 1, n_estimators)
                alpha = trial.suggest_loguniform('alpha', 0.1, 20)  # from 0 to infinite, loguniform
                reg_lambda = trial.suggest_loguniform('reg_lambda', 0.01, 20)  # from 0 to infinite, loguniform
                model = lgb.LGBMClassifier(
                    learning_rate=learning_rate,
                    boosting_type=boosting_type,
                    num_leaves=num_leaves,
                    num_threads=num_threads,
                    objective=objective,
                    metric=metric,
                    n_estimators=n_estimators,
                    min_data_in_leaf=min_data_in_leaf,
                    feature_fraction=feature_fraction,
                    bagging_fraction=bagging_fraction,
                    bagging_freq=bagging_freq,
                    max_depth=max_depth,
                    alpha=alpha,
                    reg_lambda=reg_lambda,
                    random_state=random_state
                )
                model.fit(X_train, y_train)
                preds = model.predict(X_val)

                return accuracy_score(y_val, preds)

            elif learning_rate in [1e-3, 5e-3, 1e-2]:
                min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 10, 30)
                feature_fraction = trial.suggest_uniform('feature_fraction', 0.3, 0.7)
                bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.3, 0.7)
                bagging_freq = trial.suggest_int('bagging_freq', 1, n_estimators)
                alpha = trial.suggest_loguniform('alpha', 10, 30)  # from 0 to infinite, loguniform
                reg_lambda = trial.suggest_loguniform('reg_lambda', 10, 30)  # from 0 to infinite, loguniform
                model = lgb.LGBMClassifier(
                    learning_rate=learning_rate,
                    boosting_type=boosting_type,
                    num_leaves=num_leaves,
                    num_threads=num_threads,
                    objective=objective,
                    metric=metric,
                    n_estimators=n_estimators,
                    min_data_in_leaf=min_data_in_leaf,
                    feature_fraction=feature_fraction,
                    bagging_fraction=bagging_fraction,
                    bagging_freq=bagging_freq,
                    max_depth=max_depth,
                    alpha=alpha,
                    reg_lambda=reg_lambda,
                    random_state=random_state
                )
                model.fit(X_train, y_train)
                preds = model.predict(X_val)

                return accuracy_score(y_val, preds)

            elif learning_rate in [5e-2, 1e-1]:
                min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 20, 40)
                feature_fraction = trial.suggest_uniform('feature_fraction', 0.1, 0.5)
                bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.1, 0.5)
                bagging_freq = trial.suggest_int('bagging_freq', 1, n_estimators)
                alpha = trial.suggest_loguniform('alpha', 20, 40)  # from 0 to infinite, loguniform
                reg_lambda = trial.suggest_loguniform('reg_lambda', 20, 40)  # from 0 to infinite, loguniform
                model = lgb.LGBMClassifier(
                    learning_rate=learning_rate,
                    boosting_type=boosting_type,
                    num_leaves=num_leaves,
                    num_threads=num_threads,
                    objective=objective,
                    metric=metric,
                    n_estimators=n_estimators,
                    min_data_in_leaf=min_data_in_leaf,
                    feature_fraction=feature_fraction,
                    bagging_fraction=bagging_fraction,
                    bagging_freq=bagging_freq,
                    max_depth=max_depth,
                    alpha=alpha,
                    reg_lambda=reg_lambda,
                    random_state=random_state
                )
                model.fit(X_train, y_train)
                preds = model.predict(X_val)

                return accuracy_score(y_val, preds)

        elif boosting_type == 'dart':
            n_estimators = trial.suggest_int('n_estimators', 1, 500)
            learning_rate = trial.suggest_categorical('learning_rate', [1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2,
                                                                        1e-1])  # aka eta, from 0 to 1, loguniform
            if learning_rate in [1e-4, 5e-4]:
                min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 20)
                feature_fraction = trial.suggest_uniform('feature_fraction', 0.5, 0.999)
                bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.5, 0.999)
                bagging_freq = trial.suggest_int('bagging_freq', 1, n_estimators)
                drop_rate = trial.suggest_loguniform('drop_rate', 0.01, 0.5)
                model = lgb.LGBMClassifier(
                    learning_rate=learning_rate,
                    boosting_type=boosting_type,
                    num_leaves=num_leaves,
                    num_threads=num_threads,
                    n_estimators=n_estimators,
                    objective=objective,
                    metric=metric,
                    min_data_in_leaf=min_data_in_leaf,
                    feature_fraction=feature_fraction,
                    bagging_fraction=bagging_fraction,
                    bagging_freq=bagging_freq,
                    max_depth=max_depth,
                    drop_rate=drop_rate,
                    random_state=random_state
                )
                model.fit(X_train, y_train)
                preds = model.predict(X_val)

                return accuracy_score(y_val, preds)

            elif learning_rate in [1e-3, 5e-3, 1e-2]:
                min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 10, 30)
                feature_fraction = trial.suggest_uniform('feature_fraction', 0.3, 0.7)
                bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.3, 0.7)
                bagging_freq = trial.suggest_int('bagging_freq', 1, n_estimators)
                drop_rate = trial.suggest_loguniform('drop_rate', 0.3, 0.7)
                model = lgb.LGBMClassifier(
                    learning_rate=learning_rate,
                    boosting_type=boosting_type,
                    num_leaves=num_leaves,
                    num_threads=num_threads,
                    n_estimators=n_estimators,
                    objective=objective,
                    metric=metric,
                    min_data_in_leaf=min_data_in_leaf,
                    feature_fraction=feature_fraction,
                    bagging_fraction=bagging_fraction,
                    bagging_freq=bagging_freq,
                    max_depth=max_depth,
                    drop_rate=drop_rate,
                    random_state=random_state
                )
                model.fit(X_train, y_train)
                preds = model.predict(X_val)

                return accuracy_score(y_val, preds)

            elif learning_rate in [5e-2, 1e-1]:
                min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 20, 40)
                feature_fraction = trial.suggest_uniform('feature_fraction', 0.5, 0.999)
                bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.5, 0.999)
                bagging_freq = trial.suggest_int('bagging_freq', 1, n_estimators)
                drop_rate = trial.suggest_loguniform('drop_rate', 0.5, 1)
                model = lgb.LGBMClassifier(
                    learning_rate=learning_rate,
                    boosting_type=boosting_type,
                    num_leaves=num_leaves,
                    num_threads=num_threads,
                    n_estimators=n_estimators,
                    objective=objective,
                    metric=metric,
                    min_data_in_leaf=min_data_in_leaf,
                    feature_fraction=feature_fraction,
                    bagging_fraction=bagging_fraction,
                    bagging_freq=bagging_freq,
                    max_depth=max_depth,
                    drop_rate=drop_rate,
                    random_state=random_state
                )
                model.fit(X_train, y_train)
                preds = model.predict(X_val)

                return accuracy_score(y_val, preds)

        elif boosting_type == 'rf':
            n_estimators = trial.suggest_int('n_estimators', 1, 500)
            feature_fraction_bynode = trial.suggest_loguniform('feature_fraction_bynode', 0.01, 0.999)
            if feature_fraction_bynode < 0.33:
                bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.5, 0.999)
                bagging_freq = trial.suggest_int('bagging_freq', 1, n_estimators)
                min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 0, 20)
                alpha = trial.suggest_loguniform('alpha', 0.1, 20)  # from 0 to infinite, loguniform
                reg_lambda = trial.suggest_loguniform('reg_lambda', 0.01, 20)  # from 0 to infinite, loguniform
                model = lgb.LGBMClassifier(
                    boosting_type=boosting_type,
                    num_leaves=num_leaves,
                    num_threads=num_threads,
                    feature_fraction_bynode=feature_fraction_bynode,
                    bagging_fraction=bagging_fraction,
                    bagging_freq=bagging_freq,
                    objective=objective,
                    metric=metric,
                    n_estimators=n_estimators,
                    min_data_in_leaf=min_data_in_leaf,
                    max_depth=max_depth,
                    alpha=alpha,
                    reg_lambda=reg_lambda,
                    random_state=random_state
                )
                model.fit(X_train, y_train)
                preds = model.predict(X_val)

                return accuracy_score(y_val, preds)

            elif feature_fraction_bynode > 0.33 and feature_fraction_bynode < 0.66:
                min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 10, 30)
                alpha = trial.suggest_loguniform('alpha', 10, 30)  # from 0 to infinite, loguniform
                bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.5, 0.999)
                bagging_freq = trial.suggest_int('bagging_freq', 1, n_estimators)
                reg_lambda = trial.suggest_loguniform('reg_lambda', 10, 30)  # from 0 to infinite, loguniform
                model = lgb.LGBMClassifier(
                    boosting_type=boosting_type,
                    num_leaves=num_leaves,
                    num_threads=num_threads,
                    feature_fraction_bynode=feature_fraction_bynode,
                    bagging_fraction=bagging_fraction,
                    bagging_freq=bagging_freq,
                    objective=objective,
                    metric=metric,
                    n_estimators=n_estimators,
                    min_data_in_leaf=min_data_in_leaf,
                    max_depth=max_depth,
                    alpha=alpha,
                    reg_lambda=reg_lambda,
                    random_state=random_state
                )

                model.fit(X_train, y_train)
                preds = model.predict(X_val)

                return accuracy_score(y_val, preds)

            elif feature_fraction_bynode > 0.66:
                min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 20, 40)
                alpha = trial.suggest_loguniform('alpha', 20, 40)  # from 0 to infinite, loguniform
                reg_lambda = trial.suggest_loguniform('reg_lambda', 20, 40)  # from 0 to infinite, loguniform
                bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.5, 0.999)
                bagging_freq = trial.suggest_int('bagging_freq', 1, n_estimators)
                model = lgb.LGBMClassifier(
                    boosting_type=boosting_type,
                    num_leaves=num_leaves,
                    num_threads=num_threads,
                    feature_fraction_bynode=feature_fraction_bynode,
                    objective=objective,
                    metric=metric,
                    bagging_freq=bagging_freq,
                    bagging_fraction=bagging_fraction,
                    n_estimators=n_estimators,
                    min_data_in_leaf=min_data_in_leaf,
                    max_depth=max_depth,
                    alpha=alpha,
                    reg_lambda=reg_lambda,
                    random_state=random_state
                )

                model.fit(X_train, y_train)
                preds = model.predict(X_val)

                return accuracy_score(y_val, preds)

        elif boosting_type == 'goss':
            learning_rate = trial.suggest_categorical('learning_rate', [1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2,
                                                                        1e-1])  # aka eta, from 0 to 1, loguniform
            if learning_rate in [1e-4, 5e-4]:
                min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 0, 20)
                top_rate = trial.suggest_loguniform('top_rate', 0.1, 0.5)
                model = lgb.LGBMClassifier(
                    learning_rate=learning_rate,
                    boosting_type=boosting_type,
                    num_leaves=num_leaves,
                    num_threads=num_threads,
                    objective=objective,
                    metric=metric,
                    top_rate=top_rate,
                    min_data_in_leaf=min_data_in_leaf,
                    max_depth=max_depth,
                    random_state=random_state
                )
                model.fit(X_train, y_train)
                preds = model.predict(X_val)

                return accuracy_score(y_val, preds)

            elif learning_rate in [1e-3, 5e-3, 1e-2]:
                min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 10, 30)
                top_rate = trial.suggest_loguniform('top_rate', 0.3, 0.7)
                model = lgb.LGBMClassifier(
                    learning_rate=learning_rate,
                    boosting_type=boosting_type,
                    num_leaves=num_leaves,
                    num_threads=num_threads,
                    objective=objective,
                    metric=metric,
                    top_rate=top_rate,
                    min_data_in_leaf=min_data_in_leaf,
                    max_depth=max_depth,
                    random_state=random_state
                )
                model.fit(X_train, y_train)
                preds = model.predict(X_val)

                return accuracy_score(y_val, preds)

            elif learning_rate in [5e-2, 1e-1]:
                min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 2, 40)
                top_rate = trial.suggest_loguniform('top_rate', 0.5, 0.9)
                model = lgb.LGBMClassifier(
                    learning_rate=learning_rate,
                    boosting_type=boosting_type,
                    num_leaves=num_leaves,
                    num_threads=num_threads,
                    objective=objective,
                    top_rate=top_rate,
                    min_data_in_leaf=min_data_in_leaf,
                    max_depth=max_depth,
                    random_state=random_state
                )

                model.fit(X_train, y_train)
                preds = model.predict(X_val)

                return accuracy_score(y_val, preds)

In [92]:
study = optuna.create_study(direction='minimize')
study.optimize(model_selector, n_trials=500)

[I 2023-06-11 01:16:17,326] A new study created in memory with name: no-name-153a2c30-dfcf-4ddf-ba00-c5fb7c9ae476
[I 2023-06-11 01:17:13,442] Trial 0 finished with value: 0.6706989247311828 and parameters: {'model_type': 'xgb', 'tree_method': 'hist', 'booster': 'dart', 'colsample_bytree': 0.7305749527599955, 'learning_rate': 0.0001, 'rate_drop': 0.02256162736229715, 'min_child_weight': 20}. Best is trial 0 with value: 0.6706989247311828.
[I 2023-06-11 01:17:18,238] Trial 1 finished with value: 0.6706989247311828 and parameters: {'model_type': 'xgb', 'tree_method': 'approx', 'booster': 'gbtree', 'subsample': 0.1247214377306586, 'n_estimators': 741, 'colsample_bytree': 0.2437680436994102, 'learning_rate': 0.05, 'min_split_loss': 468.4541623105902, 'min_child_weight': 22, 'alpha': 36.898939485122376, 'reg_lambda': 31.05853762195564}. Best is trial 0 with value: 0.6706989247311828.
[I 2023-06-11 01:17:27,572] Trial 2 finished with value: 0.6706989247311828 and parameters: {'model_type': 'x

[LightGBM] [Warning] bagging_fraction is set=0.9537075237001793, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9537075237001793
[LightGBM] [Warning] bagging_freq is set=48, subsample_freq=0 will be ignored. Current value: bagging_freq=48
[LightGBM] [Warning] min_data_in_leaf is set=4, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4
[LightGBM] [Warning] num_threads is set=0, n_jobs=-1 will be ignored. Current value: num_threads=0


[I 2023-06-11 01:17:53,301] Trial 3 finished with value: 0.6626344086021505 and parameters: {'model_type': 'lgbm', 'num_leaves': 420, 'boosting_type': 'rf', 'n_estimators': 195, 'feature_fraction_bynode': 0.08682816992453662, 'bagging_fraction': 0.9537075237001793, 'bagging_freq': 48, 'min_data_in_leaf': 4, 'alpha': 0.2965588998672286, 'reg_lambda': 0.13920392274270993}. Best is trial 3 with value: 0.6626344086021505.


[LightGBM] [Warning] bagging_freq is set=346, subsample_freq=0 will be ignored. Current value: bagging_freq=346
[LightGBM] [Warning] feature_fraction is set=0.4876195613812056, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4876195613812056
[LightGBM] [Warning] bagging_fraction is set=0.5136078557691223, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5136078557691223
[LightGBM] [Warning] min_data_in_leaf is set=22, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=22
[LightGBM] [Warning] num_threads is set=0, n_jobs=-1 will be ignored. Current value: num_threads=0


[I 2023-06-11 01:18:00,583] Trial 4 finished with value: 0.6666666666666666 and parameters: {'model_type': 'lgbm', 'num_leaves': 483, 'boosting_type': 'dart', 'n_estimators': 394, 'learning_rate': 0.005, 'min_data_in_leaf': 22, 'feature_fraction': 0.4876195613812056, 'bagging_fraction': 0.5136078557691223, 'bagging_freq': 346, 'drop_rate': 0.39069218976171155}. Best is trial 3 with value: 0.6626344086021505.


...

[I 2023-06-11 01:19:23,818] Trial 13 finished with value: 0.6505376344086021 and parameters: {'model_type': 'lgbm', 'num_leaves': 219, 'boosting_type': 'rf', 'n_estimators': 28, 'feature_fraction_bynode': 0.1493510376507823, 'bagging_fraction': 0.8094408757678654, 'bagging_freq': 2, 'min_data_in_leaf': 0, 'alpha': 2.72733882999385, 'reg_lambda': 0.09245029259515498}. Best is trial 13 with value: 0.6505376344086021.


[LightGBM] [Warning] bagging_fraction is set=0.7754379577879577, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7754379577879577
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=0, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=0
[LightGBM] [Warning] num_threads is set=0, n_jobs=-1 will be ignored. Current value: num_threads=0


[I 2023-06-11 01:19:24,605] Trial 14 finished with value: 0.6155913978494624 and parameters: {'model_type': 'lgbm', 'num_leaves': 88, 'boosting_type': 'rf', 'n_estimators': 6, 'feature_fraction_bynode': 0.010143208983493432, 'bagging_fraction': 0.7754379577879577, 'bagging_freq': 1, 'min_data_in_leaf': 0, 'alpha': 4.236899902301447, 'reg_lambda': 0.06904552459730254}. Best is trial 14 with value: 0.6155913978494624.
[LightGBM] [Fatal] Check failed: (config_->top_rate + config_->other_rate) <= (1.0f) at /private/var/folders/0t/816d84452dn7h591gjkg32_m0000gn/T/pip-install-r7of8ftg/lightgbm_a925e0ed4219424bac647af665e90126/compile/src/boosting/goss.hpp, line 77 .

[W 2023-06-11 01:19:24,740] Trial 15 failed with parameters: {'model_type': 'lgbm', 'num_leaves': 13, 'boosting_type': 'goss', 'learning_rate': 0.1, 'min_data_in_leaf': 12, 'top_rate': 0.9552940215382816} because of the following error: LightGBMError('Check failed: (config_->top_rate + config_->other_rate) <= (1.0f) at /private/

[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Warning] num_threads is set=0, n_jobs=-1 will be ignored. Current value: num_threads=0


LightGBMError: Check failed: (config_->top_rate + config_->other_rate) <= (1.0f) at /private/var/folders/0t/816d84452dn7h591gjkg32_m0000gn/T/pip-install-r7of8ftg/lightgbm_a925e0ed4219424bac647af665e90126/compile/src/boosting/goss.hpp, line 77 .


print('Number of finished XGB trials: {}'.format(len(xgb_study.trials)))
print('XGB Best trial:')
xgb_trial = xgb_study.best_trial

print('  Value: {}'.format(xgb_trial.value))
print('  Params: ')

for key, value in xgb_trial.params.items():
    print('    {}: {}'.format(key, value))

In [93]:
print('Number of finished study trials: {}'.format(len(study.trials)))
print('study Best trial:')
study_trial = study.best_trial

print('  Value: {}'.format(study_trial.value))
print('  Params: ')

for key, value in study_trial.params.items():
    print('    {}: {}'.format(key, value))


Number of finished study trials: 16
study Best trial:
  Value: 0.6155913978494624
  Params: 
    model_type: lgbm
    num_leaves: 88
    boosting_type: rf
    n_estimators: 6
    feature_fraction_bynode: 0.010143208983493432
    bagging_fraction: 0.7754379577879577
    bagging_freq: 1
    min_data_in_leaf: 0
    alpha: 4.236899902301447
    reg_lambda: 0.06904552459730254


In [95]:
study_best_params = study.best_params
study_best_params['random_state'] = 42
if study_best_params['model_type'] == 'lgbm':
    model = lgb.LGBMClassifier(**study_best_params)
elif study_best_params['model_type'] == 'xgb':
    model = xgb(**study_best_params)
model.fit(X_train, y_train)


LGBMClassifier(alpha=4.236899902301447, bagging_fraction=0.7754379577879577,
               bagging_freq=1, boosting_type='rf',
               feature_fraction_bynode=0.010143208983493432, min_data_in_leaf=0,
               model_type='lgbm', n_estimators=6, num_leaves=88,
               random_state=42, reg_lambda=0.06904552459730254)

In [96]:
lgb_preds = lgb_model.predict(X_val)
lgb_accuracy = accuracy_score(y_val, lgb_preds)

XGB_pred = XGB.predict(X_val)
accuracy = accuracy_score(y_val, XGB_pred)
print("\nAccuracy after tuning: %.2f%%" % (accuracy * 100.0))

In [97]:
print("-- LGB --")
print("Train ACC : %.3f" % accuracy_score(y_train, lgb_model.predict(X_train)))
print("Val ACC : %.3f" % accuracy_score(y_val, lgb_model.predict(X_val)))

-- LGB --
Train ACC : 0.924
Val ACC : 0.616


print("\n-- XGB --")
print("Train ACC : %.3f" % accuracy_score(y_train, XGB.predict(X_train)))
print("Val ACC : %.3f" % accuracy_score(y_val, XGB.predict(X_val)))

In [98]:
print(classification_report(y_val, lgb_preds))


              precision    recall  f1-score   support

           0       0.32      0.16      0.21       245
           1       0.67      0.84      0.75       499

    accuracy                           0.62       744
   macro avg       0.50      0.50      0.48       744
weighted avg       0.56      0.62      0.57       744



In [85]:
X_test = pd.get_dummies(data=test_x)
print(X_test)
lgb_preds = lgb_model.predict(test_x)

          0         1         2         3         4         5         6     \
0    -0.006672 -0.080011 -0.014580 -0.038840  0.048455 -0.040639  0.039663   
1    -0.003053 -0.011769 -0.034803 -0.041324 -0.026498 -0.014561  0.012917   
2    -0.025638 -0.016005 -0.021478  0.014413  0.048054 -0.036683 -0.034054   
3     0.028991  0.036599 -0.059128 -0.030722 -0.017587 -0.028222  0.052064   
4     0.036941 -0.039247 -0.026675 -0.006587 -0.022472 -0.016113  0.046348   
...        ...       ...       ...       ...       ...       ...       ...   
1235 -0.017606 -0.026507 -0.038503 -0.013017  0.036128 -0.006750 -0.023582   
1236 -0.001220 -0.002364 -0.025906 -0.023937  0.002847 -0.021732  0.076602   
1237  0.001026  0.023440  0.013377  0.016586  0.024960 -0.001395  0.063505   
1238  0.001634 -0.000814 -0.014695  0.003300  0.001420  0.028769 -0.025222   
1239 -0.058938  0.010668  0.004917 -0.051615  0.033669  0.037903  0.019593   

          7         8         9     ...      1526      1527    

In [99]:
sample_submission = pd.read_csv('./sample_submission.csv')
LGB_pred = pd.DataFrame(lgb_preds.astype(int))
LGB_pred = LGB_pred.rename(columns={0:'first_party_winner'})

In [87]:
LGB_pred

,first_party_winner
0,0
1,1
2,1
3,1
4,1
...,...
1235,0
1236,1
1237,1
1238,0


In [88]:
sample_submission['first_party_winner'] = LGB_pred['first_party_winner']

In [89]:
sample_submission['first_party_winner'].value_counts()

1    1058
0     182
Name: first_party_winner, dtype: int64

In [90]:
sample_submission.to_csv("./results/LGB_submission_{Train:.03f}_{Val:.03f}.csv".format(Train=accuracy_score(y_train, lgb_model.predict(X_train)), Val = accuracy_score(y_val, lgb_model.predict(X_val))), index=False)

In [ ]:
# XGB_submission = pd.read_csv('./sample_submission.csv')
# XGB_pred = pd.DataFrame(XGB_pred.astype(int))
# XGB_submission['first_party_winner'] = XGB_pred
# XGB_submission.to_csv("./Bert_XGB_submission_{Train:.03f}_{Val:.03f}.csv".format(Train=accuracy_score(y_train, XGB.predict(X_train)), Val = accuracy_score(y_val, XGB.predict(X_val))), index=False)

In [ ]:
XGB_submission

In [ ]:
LGB_submission

In [ ]:
# 3D to 2D

attention_mask_df = dlc.tensor_2_2d(train_bert_tokenized, 0)
input_ids_df = dlc.tensor_2_2d(train_bert_tokenized, 1)
token_type_ids_df = dlc.tensor_2_2d(train_bert_tokenized, 2)

attention_mask_df.info()
print('\n _______________________________ \n')
input_ids_df.info()
print('\n _______________________________ \n')
token_type_ids_df.info()


In [ ]:
# attention_mask_df.info()
attention_mask_df
# input_ids_df.info()
# input_ids_df
# token_type_ids_df.info()
# token_type_ids_df


In [ ]:
temp = pd.DataFrame()
temp = pd.concat([train_cleansed['ID'], attention_mask_df], axis=1)
temp = pd.concat([temp, input_ids_df], axis=1)
train_BertToken_df = pd.concat([temp, token_type_ids_df], axis=1)
train_BertToken_df


In [ ]:
tBTdf = so.right_merger(train_cleansed, train_BertToken_df, 0)
tBTdf